<a href="https://colab.research.google.com/github/AlexUmnov/LLM-Engineering-Essentials/blob/main/topic2/2.2_llm_workflows.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LLM Engineering Essentials by Nebius Academy

Course github: [link](https://github.com/Nebius-Academy/LLM-Engineering-Essentials/tree/main)

The course is in development now, with more materials coming soon. [Subscribe to stay updated](https://academy.nebius.com/llm-engineering-essentials/update/)

# 2.2 LLM Workflows

In Topic 1 we mostly studied how to get value from one LLM working in a single-call or a chat mode. But it's only a beginning! So much more may be achieved by orchestrating a complex workflow combining several LLM calls, tools, etc.

Orchestration will be the core idea of Topic 2. We'll guide you through:

* **LLM workflows** - **manual orchestration of several LLM calls inside one system** - in this notebook
* Orchestrated and native LLM reasoning processes in notebooks **2.3-5**
* Native tool usage and LLM agent basics in **2.6**
* LLM-powered planning and agentic systems in **2.7**

So, let's start this exciting journey!

In this notebooks, we'll discuss how to combine LLM calls in meaningful and flexible ways. We'll mostly follow the [Building Effective Agents](https://www.anthropic.com/engineering/building-effective-agents) article by Anthropic in its workflow classification - and we really recommend you to browse through it.

## Getting things ready

In [ ]:
!pip install openai -qU

In [ ]:
from google.colab import userdata
from openai import OpenAI
import os

os.environ['NEBIUS_API_KEY'] = userdata.get("nebius_api_key")

nebius_client = OpenAI(
    base_url="https://api.studio.nebius.ai/v1/",
    api_key=os.environ.get("NEBIUS_API_KEY"),
)

llama_model = "meta-llama/Llama-3.3-70B-Instruct"

def prettify_string(text, max_line_length=80):
    """Prints a string with line breaks at spaces to prevent horizontal scrolling.

    Args:
        text: The string to print.
        max_line_length: The maximum length of each line.
    """

    output_lines = []
    lines = text.split("\n")
    for line in lines:
        current_line = ""
        words = line.split()
        for word in words:
            if len(current_line) + len(word) + 1 <= max_line_length:
                current_line += word + " "
            else:
                output_lines.append(current_line.strip())
                current_line = word + " "
        output_lines.append(current_line.strip())  # Append the last line
    return "\n".join(output_lines)

def answer_with_llm(prompt: str,
                    system_prompt="You are a helpful assistant",
                    max_tokens=512,
                    client=nebius_client,
                    model=llama_model,
                    prettify=True,
                    temperature=None) -> str:

    messages = []

    if system_prompt:
        messages.append(
            {
                "role": "system",
                "content": system_prompt
            }
        )

    messages.append(
        {
            "role": "user",
            "content": prompt
        }
    )

    completion = client.chat.completions.create(
        model=model,
        messages=messages,
        max_tokens=max_tokens,
        temperature=temperature
    )

    if prettify:
        return prettify_string(completion.choices[0].message.content)
    else:
        return completion.choices[0].message.content


# Understanding LLM workflows

## Chaining

The most basic LLM workflow type is **chaining**: using several LLM calls in a sequence, the next one modifying or refining the previous ones.

<center>
<img src="
https://www.anthropic.com/_next/image?url=https%3A%2F%2Fwww-cdn.anthropic.com%2Fimages%2F4zrzovbb%2Fwebsite%2F7418719e3dab222dccb379b8879e1dc08ad34c78-2401x1000.png&w=3840&q=75" width=1000 />

[Source](https://www.anthropic.com/engineering/building-effective-agents)
</center>

Example use cases might include:

* **Localization**. Though LLMs develop towards multiliguality, it's still easier for them to answer complex instructions in English (because most of the training data, as most of web and books, is in English). A natural way of dealing with that is making a **chain**:

  * The first LLM call translates the query from the source language into English
  * The second one processes the query in English
  * The third one translates the answer back into the source language.

Before LLMs became good at structured outputs, another popular use case for chaining was (Answering the question) -> (Extracting the answer).

In many cases, however, the workflows arent' sequential, so let's discuss several more comlplex types.

## Parallelization

**Parallelization** is the workflow type where several workers process the query and their outputs are put together by an agregator to produce the final answer.

<center>
<img src="
https://www.anthropic.com/_next/image?url=https%3A%2F%2Fwww-cdn.anthropic.com%2Fimages%2F4zrzovbb%2Fwebsite%2F406bb032ca007fd1624f261af717d70e6ca86286-2401x1000.png&w=3840&q=75" width=1000 />

[Source](https://www.anthropic.com/engineering/building-effective-agents)
</center>

A special case of parallelization is what can be called **LLM MapReduce**. As an example, let's consider long document summarization. if the input is too large to be processed efficiently in one call, we can

* distribute input chunks between identical LLM workers (**map** phase)
* then ask another LLM to put summaries of individual chunks together (**recude** phase)

Another example might be **evaluation of chatbot conversations**. Usually, you want to evaluate your chatbot's proficiency along several axes: helpfulness, tone of voice etc - and all this can be scored by **LLM-as-a-Judge**. And generally it might be a good idea to score different parameters in different and parallel LLM calls - this way the prompts will be simpler and the judges' outputs more reliable. In such a system, Aggregator is optional; you can just put all the scores together without any additional LLMs.

## Routing

A customer support chatbot may have a complex, tree-like logic switching a user between several conversation branches. You might just prompt one LLM thoroughy and let it rule it out in a chat mode, but if you can describe all the scenarios, why not make things more reliable by creating a **routing** workflow?

<center>
<img src="
https://www.anthropic.com/_next/image?url=https%3A%2F%2Fwww-cdn.anthropic.com%2Fimages%2F4zrzovbb%2Fwebsite%2F5c0c0e9fe4def0b584c04d37849941da55e5e71c-2401x1000.png&w=3840&q=75" width=1000 />

[Source](https://www.anthropic.com/engineering/building-effective-agents)
</center>

In the most basic implementation, the entry-point LLM chooses between several prescribed scenarios based on the user's request. But the workflow might as well be a more complicated one:

<center>
<img src="https://drive.google.com/uc?export=view&id=1QoIL5j2C6U5u2gE_jGcF-w3e1qlgWL7d" width=600 />
</center>

Not all of the links must be other LLMs. Some might be rule-based processors or even involve human support specialists jumping in to help the client.

Another possible application of routing is choosing between a number of LLMs of various capability. For example, you could use a 8B model for simple questions or 70B model for more elaborate ones. Classifying the question's complexity might be a job for an LLM.

## Feedback loops

In complex tasks we might expect that an LLM workflow wouldn't immediately arrive at the final solution. A good example is coding: the first solution may be flawed, and one or two rounds of self-analysis and self-correction might help.

If you can describe the evaluation criteria, you can construct a **feedback loop** that would run until the evaluator gives the solution a pass or until the system hits max number of iterations.

<center>
<img src="
https://www.anthropic.com/_next/image?url=https%3A%2F%2Fwww-cdn.anthropic.com%2Fimages%2F4zrzovbb%2Fwebsite%2F14f51e6406ccb29e695da48b17017e899a6119c7-2401x1000.png&w=3840&q=75" width=1000 />

[Source](https://www.anthropic.com/engineering/building-effective-agents)
</center>

## Creating more complex workflows. Workflows vs agents

From these four primitives, you can assemble workflows of arbitrary complexity. For example, here is a potential advertisement creation workflow:

<center>
<img src="https://drive.google.com/uc?export=view&id=14DDGHYOojUClbcI59WRn5lZ_4rAWHpoW" width=600 />
</center>

LLM workflows are all human-designed. To create one, you need to come up with the process nodes and connections between them. At times, you would want something - an LLM! - to orchestrate everything for you.

Like this:

<center>
<img src="https://drive.google.com/uc?export=view&id=102GUavLMfYjqR0SftsQa5VItH2TEA0JS" width=400 />
</center>

LLM-powered orchestration makes the system into an **LLM Agent**. It's a cool and powerful thing, and we'll discuss it in more details in notebooks [A.1](https://colab.research.google.com/github/Nebius-Academy/LLM-Engineering-Essentials/blob/main/topic2/a.1_llm_tools_and_agents.ipynb) and [A.2](https://colab.research.google.com/github/Nebius-Academy/LLM-Engineering-Essentials/blob/main/topic2/a.1_llm_tools_and_agents.ipynb). However, it makes things less transparent and less reliable in comparison with manually orchestrated pipelines.

In the rest of this notebook, we'll work out several particular examples of LLM orchestration: summarization and localization.

# LLM workflow examples

## Summarization

Let's try to write a simple LLM summarization script. As an example text we'll take an article from Wikipedia about paws.

Note: we'll take a different model here, specifically **deepseek-ai/DeepSeek-R1** because it's tokenizer doesn't have additional license requirements.

In [ ]:
import requests, bs4

content = requests.get("https://en.wikipedia.org/wiki/Paw").content
parsed = bs4.BeautifulSoup(content)
content_div = parsed.find("div", "mw-content-container")
full_text = content_div.get_text()

We are using **BeautifulSoup** to parse out the contents of the page omitting everything except for the main text.

Now let's write a simple llm summarization code:

In [ ]:
model = "deepseek-ai/DeepSeek-R1"

def summarize_with_llm(text):
    chat_completion = nebius_client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": f"Summarize the most important aspects of the following text: {text}. Try to be short."}]
    )
    return chat_completion.choices[0].message.content

In [ ]:
summarize_with_llm(full_text)

This looks like a good summary of the article. However, this is the simplest example. Let's try to do something a bit more complicated.

## Map reduce summarization

In [ ]:
def get_wiki_text(url):
    content = requests.get(url).content
    parsed = bs4.BeautifulSoup(content)
    content_div = parsed.find("div", "mw-content-container")
    full_text = content_div.get_text()
    return full_text

A page on *2023 in American television* is considered one of the longest pages on Wikipedia. It's mostly long because it's a list of all shows released that year with descriptions. However, it's great for us to test our long text summarization skills.

In [ ]:
full_text = get_wiki_text("https://en.wikipedia.org/wiki/2023_in_American_television")

The length of the text is not that interesting to us as the number of tokens. Let's try to look at both.

We'll use **huggingface** to get the model's tokenizer.

In [ ]:
from transformers import AutoTokenizer


def get_token_count(text):
    encoding = AutoTokenizer.from_pretrained(model)

    encoded = encoding.encode(text)
    return len(encoded)

In [ ]:
print(f"Number of tokens: {get_token_count(full_text)}")
print(f"Number of characters: {len(full_text)}")

Even though technically DeepSeek R1 can take this whole text at once (it has 164000 token-long context window), it's usually not ideal. Especially because for models computations grow more than linearly proportionally to length. So it's better to summarize in **MapReduce** style, where we summarize small parts and then generate a summary for the whole text. It can also help us summarize texts which are larger than our context window.

Let's experiment with both:

In [ ]:
naive_summary = summarize_with_llm(full_text)
naive_summary

To orchestrate a MapReduce pipeline, we'll need to break text down into chunks - and we don't want it to be sliced mid-sentence. So, we'll use special tools for that.

Langchain has a handy tool called `TextSplitter` which allows to split a text following specific rules.

For example, `RecursiveCharacterTextSplitter` can split texts recursively based on list of characters until it reaches desired length. It also allows you to set up overlap so that chunks have some connections between each other. It's a useful thing, but we will not be using this here.

The default delimiter list is `["\n\n", "\n", " ", ""]`. Which in theory gives us splitting by paragraphs, subparagraphs, words and then characters.

Langchain also allows you to define length functions for Splitters, which will be used to determine if the chunk is of an appropriate length. We can even instantiate a length function from `tiktoken` encoder directly, so that our chunk length is tied to the token count.

In [ ]:
!pip install -qU langchain langchain-openai

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
    tokenizer=AutoTokenizer.from_pretrained(model),
    chunk_size=10000,
    chunk_overlap=0
)

In [ ]:
splitted_text = splitter.split_text(full_text)
len(splitted_text)

Let's create our Map and Reduce operations.

Notice that langchain uses a bit of a different notation for it's chains.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOpenAI(
    model=llama_model,
    base_url="https://api.studio.nebius.ai/v1/",
    api_key=os.environ['NEBIUS_API_KEY']
)

map_prompt = ChatPromptTemplate.from_messages(
    [("human", "Write a concise summary of the following:\\n\\n{context}")]
)

map_chain = map_prompt | llm | StrOutputParser()


reduce_template = """
The following is a set of summaries:
{docs}
Take these and distill it into a final, consolidated summary
of the main themes.
"""

reduce_prompt = ChatPromptTemplate([("human", reduce_template)])

reduce_chain = reduce_prompt | llm | StrOutputParser()

Now, in the simplest form our MapReduce summarization would look like this:

In [ ]:
from tqdm.auto import tqdm

In [ ]:
def map_reduce_summarization(docs):
    summaries = map(
        lambda doc: map_chain.invoke({"context": doc}),
        tqdm(docs)
    )

    final_summary = reduce_chain.invoke({"docs": "\n\n".join(summaries)})
    return final_summary

In [ ]:
map_reduce_summarization(splitted_text)

## Map reduce with LLM orchestration

In some cases we might need a bit more complicated orchestration of MapReduce calls. Let's imagine an example scenario.

We want to create a party of adventurers. We can decompose this task into multiple steps. Namely:

1. Generate a rought outline of the party, amount of members, rough descriptions.
2. For each member generate full story and skill list
3. Gather all descriptions and generate a short story of how they came to be together.

<center>
<img src="https://drive.google.com/uc?export=view&id=1ScixZwB5AK9TQWrpIg5nTErT_M0Y-S9C" width=600 />
</center>

For part 2 we can reuse our previous example from the structured generation notebook.

In [ ]:
from typing import List
from pydantic import BaseModel

class CharacterProfile(BaseModel):
    name: str
    age: int
    special_skills: List[str]
    traits: List[str]
    character_class: str
    origin: str

def generate_character(description: str):
    completion = nebius_client.beta.chat.completions.parse(
        model=llama_model,
        messages=[
            {
                "role": "user",
                "content": "Design a role play character based on the following"\
                          f"short description {description}"}
        ],
        extra_body={
            "guided_json": CharacterProfile.model_json_schema()
        }
    )

    return CharacterProfile.model_validate_json(completion.choices[0].message.content)

In [ ]:
generate_character("Dwarf druid")

Now, for steps 1 and 3:

In [ ]:
import json

def pregenerate_party():
    json_output = nebius_client.chat.completions.create(
        messages=[{
            'role': 'user', \
            'content': 'Generate a short description for a party of adventurers.\n'\
            'A party should have 3-5 adventures and a balanced classes set, i.e. '\
            'have at least a melee tank, a support and a damage dealer. \n'\
            'Output those short descriptions in a json format as a list with the key "party".\n'\
            'Each description should be a string with only a couple of details'

        }],
        model=llama_model,
        response_format={"type": "json_object"}
    ).choices[0].message.content
    return json.loads(json_output)['party']

In [ ]:
pregenerate_party()

In [ ]:
def generate_back_story(party_details: str):
    return nebius_client.chat.completions.create(
        messages=[
            {
                'role': 'user', \
                'content': 'Based on the following party details generate '\
                'a short story of how this party came to be together.\n'
                f'{str(party_details)}'
            }
        ],
        model=llama_model,
    ).choices[0].message.content

Now to put it all together in a workflow:

In [ ]:
def generate_party():
    party = pregenerate_party()

    character_sheets = [
        generate_character(character)
        for character in party
    ]

    backstory = generate_back_story(character_sheets)

    character_sheets_str = "\n".join([
        str(character) for character in character_sheets
    ])

    return f"""
Party description:
{json.dumps(party, indent=4)}

Party backstory:
{backstory}

Party members:
{character_sheets_str}
"""

In [ ]:
print(generate_party())

## LLM localization

Even if your base LLM is better at English that at your target language, you can easily translate your outputs with another LLM call.

In [ ]:
def translate_to_language(input: str, target_language: str):
    return nebius_client.chat.completions.create(
        messages=[
            {
                'role': 'user', \
                'content': f'Translate the following text into {target_language}:\n{input}'
            }
        ],
        model=llama_model,
    ).choices[0].message.content

In [ ]:
party = generate_party()
print(party)
translated_party = translate_to_language(party, "Spanish")
print(translated_party)

Let's also try asking the LLM to generate the party in Spanish, this omitting the translation stage:

In [ ]:
def pregenerate_party_in_language(target_language: str):
    json_output = nebius_client.chat.completions.create(
        messages=[{
            'role': 'user', \
            'content': 'Generate a short description for a party of adventurers.\n'\
            'A party should have 3-5 adventures and a balanced classes set, i.e. '\
            'have at least a melee tank, a support and a damage dealer. \n'\
            'Output those short descriptions in a json format as a list with the key "party".\n'\
            'Each description should be a string with only a couple of details.\n'\
            f'Generate in {target_language}'
        }],
        model=llama_model,
        response_format={"type": "json_object"}
    ).choices[0].message.content
    return json.loads(json_output)['party']

In [ ]:
pregenerate_party_in_language("Dutch")

# Practice tasks


##  Task 1. Character localization

Let's add localization to our simple chat NPC class from Topic 1.

Your task will be to implement the following localized chat pipeline:
- The user's input is translated into English,
- The NPC answers an English query in English (already implemented)
- The NPC's answer is translated into the target language, and the translation is returned to the user.

Here's all the code for character creation we used before. Add new code in necessary places

In [ ]:
from collections import defaultdict, deque
from openai import OpenAI
from typing import Dict, Any, Optional
import datetime
import string
import random
from dataclasses import dataclass

@dataclass
class NPCConfig:
    world_description: str
    character_description: str
    history_size: int = 10
    has_scratchpad: bool = False

class NPCFactoryError(Exception):
    """Base exception class for NPC Factory errors."""
    pass

class NPCNotFoundError(NPCFactoryError):
    """Raised when trying to interact with a non-existent NPC."""
    def __init__(self, npc_id: str):
        self.npc_id = npc_id
        super().__init__(f"NPC with ID '{npc_id}' not found")

class SimpleChatNPC:
    def __init__(self, client: OpenAI, model: str, config: NPCConfig):
        self.client = client
        self.model = model
        self.config = config
        self.chat_histories = defaultdict(lambda: deque(maxlen=config.history_size))

    def get_system_message(self) -> Dict[str, str]:
        """Returns the system message that defines the NPC's behavior."""
        character_description = self.config.character_description

        if self.config.has_scratchpad:
            character_description += """
You can use scratchpad for thinking before you answer: whatever you output between #SCRATCHPAD and #ANSWER won't be shown to anyone.
You start your output with #SCRATCHPAD and after you've done thinking, you #ANSWER"""

        return {
            "role": "system",
            "content": f"""WORLD SETTING: {self.config.world_description}
###
{character_description}"""
        }

    def chat(self, user_message: str, user_id: str) -> str:
        """Process a user message and return the NPC's response."""
        messages = [self.get_system_message()]

        # Add conversation history
        history = list(self.chat_histories[user_id])
        if history:
            messages.extend(history)

        # Add new user message
        user_message_dict = {
            "role": "user",
            "content": user_message
        }
        self.chat_histories[user_id].append(user_message_dict)
        messages.append(user_message_dict)

        try:
            completion = self.client.chat.completions.create(
                model=self.model,
                messages=messages,
                temperature=0.6
            )

            response = completion.choices[0].message.content

            # Handle scratchpad if enabled
            response_clean = response
            if self.config.has_scratchpad:
                import re
                scratchpad_match = re.search(r"#SCRATCHPAD(:?)(.*?)#ANSWER(:?)", response, re.DOTALL)
                if scratchpad_match:
                    response_clean = response[scratchpad_match.end():].strip()


            # Store response in history, including the scratchpad
            self.chat_histories[user_id].append({
                "role": "assistant",
                "content": response
            })

            # Return the message to the user without a scratchpad
            return response_clean

        except Exception as e:
            return f"Error: {str(e)}"

class NPCFactory:
    def __init__(self, client: OpenAI, model: str):
        self.client = client
        self.model = model
        self.npcs: Dict[str, SimpleChatNPC] = {}
        self.user_ids: Dict[str, str] = {}  # username -> user_id mapping

    def generate_id(self) -> str:
        """Generate a random unique identifier."""
        return ''.join(random.choice(string.ascii_letters) for _ in range(8))

    def register_user(self, username: str) -> str:
        """Register a new user and return their unique ID.
        If username already exists, appends a numerical suffix."""
        base_username = username
        suffix = 1

        # Keep trying with incremented suffixes until we find an unused name
        while username in self.user_ids:
            username = f"{base_username}_{suffix}"
            suffix += 1

        user_id = self.generate_id()
        self.user_ids[username] = user_id
        return user_id

    def set_user_language(self, user_id: str, language: str):
        """Set the preferred language for a user."""
        ### YOUR CODE HERE

    def register_npc(self, world_description: str, character_description: str,
                     history_size: int = 10, has_scratchpad: bool = False) -> str:
        """Create and register a new NPC, returning its unique ID."""
        npc_id = self.generate_id()

        config = NPCConfig(
            world_description=world_description,
            character_description=character_description,
            history_size=history_size,
            has_scratchpad=has_scratchpad
        )

        self.npcs[npc_id] = SimpleChatNPC(self.client, self.model, config)
        return npc_id

    def chat_with_npc(self, npc_id: str, user_id: str, message: str) -> str:
        """Send a message to a specific NPC from a specific user.

        Args:
            npc_id: The unique identifier of the NPC
            user_id: The unique identifier of the user
            message: The message to send

        Returns:
            The NPC's response

        Raises:
            NPCNotFoundError: If the specified NPC doesn't exist
        """
        if npc_id not in self.npcs:
            raise NPCNotFoundError(npc_id)

        npc = self.npcs[npc_id]
        return npc.chat(message, user_id)

    def get_npc_chat_history(self, npc_id: str, user_id: str) -> list:
        """Retrieve chat history between a specific user and NPC.

        Args:
            npc_id: The unique identifier of the NPC
            user_id: The unique identifier of the user

        Returns:
            List of message dictionaries containing the chat history

        Raises:
            NPCNotFoundError: If the specified NPC doesn't exist
        """
        if npc_id not in self.npcs:
            raise NPCNotFoundError(npc_id)

        return list(self.npcs[npc_id].chat_histories[user_id])

In [ ]:
from openai import OpenAI

# Nebius uses the same OpenAI() class, but with additional details
client = OpenAI(
    base_url="https://api.studio.nebius.ai/v1/",
    api_key=os.environ.get("NEBIUS_API_KEY"),
)

model = "meta-llama/Meta-Llama-3.1-405B-Instruct"

# Creating a factory
npc_factory = NPCFactory(client=client, model=model)

In [ ]:
# Register a user
user_id = npc_factory.register_user("Alice")

# Set user preffered language
preffered_language = "Old English"
npc_factory.set_user_language(user_id, preffered_language)

# Create an NPC
npc_id = npc_factory.register_npc(
    world_description="Medieval London, XIII century",
    character_description="A knight at Edward I's court",
    has_scratchpad=False
)

In [ ]:
def prettify_string(text, max_line_length=80):
    """Prints a string with line breaks at spaces to prevent horizontal scrolling.

    Args:
        text: The string to print.
        max_line_length: The maximum length of each line.
    """

    output_lines = []
    lines = text.split("\n")
    for line in lines:
        current_line = ""
        words = line.split()
        for word in words:
            if len(current_line) + len(word) + 1 <= max_line_length:
                current_line += word + " "
            else:
                output_lines.append(current_line.strip())
                current_line = word + " "
        output_lines.append(current_line.strip())  # Append the last line
    return "\n".join(output_lines)

In [ ]:
# We can hack our own code a bit and use translation features to test the system.

npc = npc_factory.npcs[npc_id]

message = "Hello, who are you and what brings you here?"
message_translated = # use localization feature of your NPC to translate the message
print(f"Translated message: {message_translated}")

response = npc_factory.chat_with_npc(
    npc_id,
    user_id,
    message_translated
)
print("Original answer")
print(prettify_string(response))
print("Translated answer")
print(prettify_string(npc.localize_input(response)))

## Task 2. Translating poetry

LLMs are notoriously bad at translating poetry. The resulting poems rarely have good rhyme and rhytm. Let's try to naively translate some Humpty Dumpty rhyme to a language of your choice:

In [ ]:
language = "Dutch"
poem = """Humpty Dumpty sat on a wall,
Humpty Dumpty had a great fall.
All the king's horses and all the king's men
Couldn't put Humpty together again."""

print(answer_with_llm(
    f"Translate the following children's rhyme to {language}\n{poem}"
))

This is hardly a good translation because the rhyme is completely broken.

You task is to create a chain of LLM calls to do the following steps in translating a poem:

1. Do a naive literal translation to preserve the meaning
2. Rewrite the translation to retain rhyme and rhythm but perhaps loosing a bit of meaning.
3. Finally have an editor look at both original and translation and make final touch ups.

We encourage you to try and prompt your LLMs to do the job of "Translator", "Editor" and so on.
Also choose the language you can understand best so that you can evaluate the result well (you can also change the original to some other language)

In [ ]:
def translate_in_stages(input: str, language: str) -> str:
    pass

In [ ]:
translate_in_stages(poem, language)

## Task 3. Finding a hero

Our kingdom has a very formal process for approving a hero for a specific quest.
You task is to implement the approvement process using LLM calls:

You receive a request for a hero and a description of a hero to hire for this quest.

**Step 1**. Check that request is formally correct. You can come up with your own ideas, but we suggest the following criteria:
- It has a name of the person requesting a hero and a date;
- It has a description of who's going to supply the hero with money and other resources;
- It has a reason why the hero is needed, some quest or challenge;
- It has a recommended qualification for the hero;

**Step 2**. Check that the problem with which the request is trying to deal is sufficient to actually find a hero, or perhaps an author might do it themself or find an easier solution.

**Step 3**. Make sure that the description of the hero is compatible with the quest and requirements placed on the hero.

These steps should be performed sequentially, one after another. If any of the stages fail, immediately return a refusal with justification - you don't want to waste any more compute on unworthy queries! If all the three steps succeed, return "accepted".

In [ ]:
def check_hero_request(request_for_a_hero: str, hero_descripition: str) -> str:
    pass

In [ ]:
epic_request = """
Epic Hero Request
Name of Requestor: Lord Aeldric of the Silver Vale
Date: The 15th Day of Bloomrise, Year 1025 of the Dawnstar Calendar

Resource Provision:
The hero shall be provisioned by the Guild of Eternal Flame, a conclave of wealthy artificers and arcane financiers, who have pledged a sum of 500,000 gold crowns, enchanted arms and armor, rare tomes of forgotten magic, a sky-bound griffon steed, and a personal aide skilled in healing and reconnaissance. All resources shall be delivered at the Hall of Summoning in Ironhold.

Purpose of Request / Quest Description:
Darkness stirs in the Hollow Spine Mountains, where the Obsidian Serpent — an ancient beast thought long dead — has risen anew. Villages lie in ruin, and the skies turn black with ash. The hero is summoned to descend into the Abyssal Breach, recover the lost Emberheart Crystal, and seal the rift before the World Spine fractures and all realms fall into chaos.

Recommended Hero Qualifications:

Proven mastery in combat, both arcane and martial

Experience in surviving extreme environments and demonic incursions

Wisdom enough to resist corruption, and strength to slay without hesitation

Familiarity with ancient dialects and lost technologies

A heart unwavering in the face of despair, and a spirit unbreakable by shadow

Let the stars guide the right soul to answer. The fate of the realms balances on a blade’s edge.
"""

not_epic_request = """
Epic Hero Request
Name of Requestor: Steve
Date: 3rd of January 2025

Resource Provision:
I'll pay from my pocket

Quest Description:
I need someone to run to a supermarket for me, i'm hungry

Required qualification:
- Be very fast
- Be smart to buy good snacks.
"""

wrong_request = """
Hello, I need a mighty warrior to slay evil, thank you!
"""

In [ ]:
epic_hero = """
Hero Profile: Kaelen Thorne, the Ash-Wrought Sentinel

Forged in the fires of the Blistering Wars and tempered by years wandering the haunted ruins of the Old Kingdoms, Kaelen Thorne is a battle-scarred veteran clad in rune-etched obsidian armor. With one eye gifted by the Seers of Valemire—able to glimpse the truth behind illusions—and a blade forged from a fallen star, Kaelen walks the line between light and shadow.

Equal parts scholar and warrior, Kaelen speaks the tongues of forgotten realms and wields spells that twist the very air. Haunted but unyielding, Kaelen has turned away crowns and glory before—but for a quest that may decide the fate of all creation, the Sentinel rises once more.
"""

not_so_epic_hero = """
Tom the cat

Is a cat, supposed to catch mice, but can't really do it.
Has a lot of different surprising weapons and contraptions, but they always work against them.

Works for cat food.
"""

In [ ]:
check_hero_request(request_for_a_hero=epic_request, hero_descripition=epic_hero)

In [ ]:
check_hero_request(request_for_a_hero=epic_request, hero_descripition=not_so_epic_hero)

In [ ]:
check_hero_request(request_for_a_hero=not_epic_request, hero_descripition=epic_hero)

In [ ]:
check_hero_request(request_for_a_hero=wrong_request, hero_descripition=epic_hero)